In [76]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET

url = "https://rcerijswijk.adlibhosting.com/api.wo2/wwwopac.ashx?database=collect&limit=100&search=all"
response = requests.get(url)

root = ET.fromstring(response.content)

In [77]:
def select_images(association_obj):
    img_prefix = "https://images.memorix.nl/rce/thumb/1600x1600/"
    img_postfix = ".jpg"
    
    data = []

    for record in root.iter("record"):
        associations = [association.text for association in record.findall(
        ".//Associated_subject/association.subject")]
        if association_obj in associations:
            uuids = [reproduction_ref.text for reproduction_ref in record.findall(
            ".//Reproduction/reproduction.reference")]
            association_name = association_obj
            for uuid in uuids:
                img_url = img_prefix + uuid + img_postfix
                data.append({
                    "Association": association_name,
                    "Url": img_url
                })
        else:
            continue
    
    pd.set_option('display.max_colwidth', None)
    df = pd.DataFrame(data)
    return df
    
select_images("meubel")

,Association,Url
0,meubel,https://images.memorix.nl/rce/thumb/1600x1600/720fad26-44d0-f8f1-a651-a5d89f0e5f18.jpg
1,meubel,https://images.memorix.nl/rce/thumb/1600x1600/ae1104f1-ad64-4c31-d61d-6b2b063547d8.jpg
2,meubel,https://images.memorix.nl/rce/thumb/1600x1600/f3dc5b01-16a2-e087-8059-2d1ee7fb31a0.jpg
3,meubel,https://images.memorix.nl/rce/thumb/1600x1600/a2e41c72-c95a-f9b6-4633-13fb2999b0f8.jpg
4,meubel,https://images.memorix.nl/rce/thumb/1600x1600/21d7bfe7-7966-7b35-2b5b-1ba3dec5914a.jpg
5,meubel,https://images.memorix.nl/rce/thumb/1600x1600/96c54eee-1747-1cd1-992d-0a6092376471.jpg
6,meubel,https://images.memorix.nl/rce/thumb/1600x1600/971d1b94-ad8f-7750-18b1-3fcff4bc73e6.jpg
7,meubel,https://images.memorix.nl/rce/thumb/1600x1600/506ea4ad-2d19-39d7-312a-d86c255c705e.jpg
8,meubel,https://images.memorix.nl/rce/thumb/1600x1600/11a8922f-1ec7-1c90-3e4f-8af6b62b7d9c.jpg
9,meubel,https://images.memorix.nl/rce/thumb/1600x1600/cab79861-4dc7-3c02-eb05-fdaecc550fcf.jpg
